In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genaidata/train.json
/kaggle/input/genaidata/test.json


In [2]:
!pip install simpletransformers

  Obtaining dependency information for simpletransformers from https://files.pythonhosted.org/packages/26/9d/1030f946de5a9c312586b05c881194153fa34b54ca869dd9cba6a2bbad97/simpletransformers-0.64.5-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/d3/96/9251b421d0a1c7d625a82a04bea56b8a9830c785940ec16db454b85c6db7/streamlit-1.29.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for validators<1,>=0.2 from https://files.pythonhosted.org/packages/3a/0c/785d317eea99c3739821718f118c70537639aa43f96bfa1d83a71f68eaf6/validators-0.22.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.7 M

In [3]:
#importing required libraries
import json
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer
import pandas as pd

In [4]:
#extracting training data from the provided json file
with open('/kaggle/input/genaidata/train.json', 'r') as file:
    data = json.load(file)


questions = []
contexts = []
answers= []

for document in data['data']:
  # paragraphs.append(document)
  for paragraph in document['paragraphs']:
        for qa in paragraph['qas']:
            questions.append(qa['question'])
            contexts.append(paragraph['context'])
            if qa['answers']:
                answers.append(qa['answers'][0]['text'])
            else:
                answers.append("")

df = pd.DataFrame({
    'question': questions,
    'context' : contexts,
    'answer' : answers
})

In [5]:

#defining the functiont convert the dataframe rows to required format
def convert_to_squad_format(row):
    qas = [
        {
            "id": row.name,
            "is_impossible": row['answer'] is None,
            "question": row['question'],
            "answers": [{"text": str(row['answer']), "answer_start": str(row['context']).find(str(row['answer']))}]
        }
    ]
    return {"context": str(row['context']), "qas": qas}



In [6]:
#obtaining training data
train_data = df.apply(convert_to_squad_format, axis=1).tolist()


#obtaining test data
with open('/kaggle/input/genaidata/train.json', 'r') as file:
    data = json.load(file)
questions2 = []
contexts2 = []
answers2= []

for document in data['data']:
  # paragraphs.append(document)
  for paragraph in document['paragraphs']:
        for qa in paragraph['qas']:
            questions2.append(qa['question'])
            contexts2.append(paragraph['context'])
            if qa['answers']:
                answers2.append(qa['answers'][0]['text'])
            else:
                answers2.append("")

df2 = pd.DataFrame({
    'question': questions2,
    'context' : contexts2,
    'answer' : answers2
})
test_data = df2.apply(convert_to_squad_format, axis=1).tolist()

In [7]:
import pandas as pd
from simpletransformers.question_answering import QuestionAnsweringModel

# Initialize the model
model_type = "distilbert"
model_name = "distilbert-base-cased-distilled-squad"
model_args = {"train_batch_size": 16, "num_train_epochs": 10, "evaluate_during_training": True}
model = QuestionAnsweringModel(model_type, model_name, args=model_args)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [8]:
import pandas as pd
from simpletransformers.question_answering import QuestionAnsweringModel

#Model using Distilbert-base-cased-distilled-squad
model_type = "distilbert"
model_name = "distilbert-base-cased-distilled-squad"
model_args = {"train_batch_size": 16, "num_train_epochs": 10, "evaluate_during_training": True}
model = QuestionAnsweringModel(model_type, model_name, args=model_args)


In [9]:
#training the model on our own dataset
model.train_model(train_data, eval_data=test_data, output_dir="output/")

add example index and unique id: 100%|██████████| 80/80 [00:00<00:00, 278460.02it/s]


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/11 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 80/80 [00:02<00:00, 38.16it/s]

add example index and unique id: 100%|██████████| 80/80 [00:00<00:00, 223994.87it/s]


Running Evaluation:   0%|          | 0/22 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/22 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/11 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 80/80 [00:01<00:00, 42.22it/s]

add example index and unique id: 100%|██████████| 80/80 [00:00<00:00, 302292.18it/s]


Running Evaluation:   0%|          | 0/22 [00:00<?, ?it/s]

(110,
 {'global_step': [11, 22, 33, 44, 55, 66, 77, 88, 99, 110],
  'correct': [36, 45, 52, 56, 59, 60, 60, 60, 60, 60],
  'similar': [43, 33, 24, 20, 21, 18, 19, 20, 20, 20],
  'incorrect': [1, 2, 4, 4, 0, 2, 1, 0, 0, 0],
  'train_loss': [0.9235565066337585,
   0.1513584852218628,
   0.017088621854782104,
   0.12991735339164734,
   0.6869739294052124,
   0.01286843977868557,
   0.0019219457171857357,
   0.001060581300407648,
   0.004748880863189697,
   0.0009534836281090975],
  'eval_loss': [-5.656427556818182,
   -7.827769886363637,
   -9.397904829545455,
   -9.605113636363637,
   -10.811079545454545,
   -10.669034090909092,
   -10.673295454545455,
   -10.888494318181818,
   -11.037997159090908,
   -11.059659090909092]})

In [10]:
#extracting answers and probabilities via model
predictions, raw_outputs = model.predict(test_data)

add example index and unique id: 100%|██████████| 80/80 [00:00<00:00, 223547.18it/s]


Running Prediction:   0%|          | 0/22 [00:00<?, ?it/s]

In [11]:
for i, pred in enumerate(predictions):
    question = test_data[i]['qas'][0]['question']
    predicted_answer = next((ans for ans in pred['answer'] if ans != ''), pred['answer'][1])
#     print(f"Question {i + 1}: {question}")
#     print(f"Predicted Answer: {predicted_answer}")
#     print()

In [12]:
answers = df2['answer']

In [13]:
from sklearn.metrics import f1_score
ground_truth_answers = answers
predicted_answers = [next((ans for ans in pred['answer'] if ans != ''), pred['answer'][1]) for pred in predictions]
f1 = f1_score(ground_truth_answers, predicted_answers, average='micro')

print(f"F1 Score: {f1}")

F1 Score: 0.3375
